In [1]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [9]:
#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd
import numpy as np

In [10]:
#HMP dataset
text_file = "/Users/shwetakinger/Desktop/ERSP_data/merged_bmi_mapping_final__original_study_HMP__.txt"
biom_file = "/Users/shwetakinger/Desktop/newtables/HMP.biom"


In [11]:
m = pd.read_csv(text_file, sep="\t")

In [12]:
m.columns

Index(['#SampleID', 'BarcodeSequence', 'LinkerPrimerSequence', 'BMI',
       'bmi_group_binned', 'bmi_group_coded', 'original_study',
       'combined_study_bmi_group', 'PCR_PRIMERS', 'TARGET_SUBFRAGMENT', 'AGE',
       'ELEVATION', 'LONGITUDE', 'COUNTRY', 'SEQUENCING_METH', 'SAMPLE_CENTER',
       'Description_duplicate', 'ReversePrimer', 'COLLECTION_DATE', 'SEX',
       'FAMILY_RELATIONSHIP_GG', 'STUDY_CENTER', 'EXPERIMENT_CENTER',
       'bmi_group_amish', 'RUN_CENTER', 'LATITUDE', 'Description'],
      dtype='object')

In [13]:
m.index

RangeIndex(start=0, stop=208, step=1)

In [ ]:
#load biom table
b= biom.load_table(biom_file)

In [15]:
m['help'] = m['#SampleID'].apply(str)
m.set_index('help', inplace=True)

In [18]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
#print(i)
i_2 = m[m.bmi_group_coded != "Overweight"]
#print(i_2)
#i_2.loc[i_2.bmi_group_binned == 1, 'bmi_group_binned'] = 0
i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0
#i_2[i_2.bmi_group_binned == 1]


/Users/shwetakinger/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
#get biom file
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
#print(p)

In [ ]:
c = p.loc[i, :]
#print(c)
indices = pd.isnull(c).any(1).nonzero()[0]
print(indices)
#drop columns with NaN
c[pd.isnull(c).any(axis=1)]
c = c.dropna(how='any')  
#print(c)
#c.sum(axis=1)

In [22]:
#eliminate samples that have been removed from metadata
#print(i_2.bmi_group_binned.shape)
i_2 = i_2.drop(i_2.index[indices])
#i_2[i_2.bmi_group_binned == 0]
#print(i_2.shape)
#print(i_2.bmi_group_binned)

In [ ]:
#print(i_2.shape)
i_2[i_2.bmi_group_binned == 1]

In [24]:
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

/Users/shwetakinger/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/shwetakinger/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [25]:
from sklearn.svm import SVC
from operator import itemgetter
import numpy as np
#run Grid search with SVC algorithm
params = [
    {'C': [1], 'kernel': ['linear']},
    
]
clf = GridSearchCV(SVC(random_state=1), param_grid=params, cv=10, scoring='roc_auc')
clf.fit(c.fillna(0).as_matrix(), i_2.bmi_group_binned)
print(clf.best_score_)
print(clf.best_params_)
print(clf.best_estimator_)
print(clf.grid_scores_)
top_score = sorted(clf.grid_scores_, key=itemgetter(1), reverse=True)[:1]
for score in top_score:
    print(np.std(score.cv_validation_scores)) 

0.5924021234240213
{'kernel': 'linear', 'C': 1}
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)
[mean: 0.59240, std: 0.17119, params: {'kernel': 'linear', 'C': 1}]
0.171193683641


In [26]:
from sklearn.tree import DecisionTreeClassifier
params = {
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf':[3,4,5,6],
    'max_features': ['sqrt'],
    'max_depth':[3,4,5],
    'min_samples_split': [2,3,4,5],
}
clf = GridSearchCV(DecisionTreeClassifier(random_state=1), params, cv=10, scoring='roc_auc')
clf.fit(c.fillna(0).as_matrix(), i_2.bmi_group_binned)
print(clf.best_score_)
print(clf.best_params_)
print(clf.best_estimator_)
clf = clf.best_estimator_
#clf = DecisionTreeClassifier(max_depth=4)
#clf.fit(c.fillna(0).as_matrix(), i_2.bmi_group_binned)
importances = clf.feature_importances_
print(importances)



0.5947246184472461
{'max_depth': 4, 'max_features': 'sqrt', 'criterion': 'gini', 'min_samples_leaf': 5, 'min_samples_split': 2}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')
[ 0.  0.  0. ...,  0.  0.  0.]


In [27]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(c.fillna(0).as_matrix(), i_2.bmi_group_binned)
importances = clf.feature_importances_
print('RandomF')
print(importances)


RandomF
[ 0.          0.          0.00024372 ...,  0.          0.00017368  0.        ]


In [ ]:
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [ ]:
paramsRandomForest = {
    'n_estimators':[20,50,100],
    'max_depth': [None, 2,3,4,5],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2,3,5],
    'min_samples_leaf':[2,4,5,]
    
}
clf = GridSearchCV(RandomForestClassifier(random_state=1, n_jobs=2), paramsRandomForest, cv=10, scoring='roc_auc')
clf.fit(c.fillna(0).as_matrix(), i_2.bmi_group_binned)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, random_state=1)
cross_val_score(clf, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc').mean()

In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(max_features='sqrt', n_estimators=1000)
    #cross-validation AUC score
    scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))